# Advent of Code 2022
## [Day 7: No Space Left On Device](https://adventofcode.com/2022/day/7)

In [1]:
import aocd
input_data = aocd.get_data(year=2022, day=7).split("\n")
input_data[:10]

['$ cd /',
 '$ ls',
 'dir dpbwg',
 'dir dvwfscw',
 'dir hccpl',
 'dir jsgbg',
 'dir lhjmzsl',
 '63532 mwvbpw.mmg',
 '239480 npj',
 'dir pngs']

### Part 1

In [2]:
from pathlib import Path, PurePosixPath

In [3]:
(PurePosixPath('/') / 'foo' / '..')

PurePosixPath('/foo/..')

In [4]:
input_data[15][5:]

'dpbwg'

In [5]:
class inode:
    path: Path
    nodetype: str = "dir"
    children: list
    size: int = 0
    
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        self.children = []
    
    def total_size(self):
        return self.size + sum([i.total_size() for i in self.children])
    
    def __repr__(self):
        if self.nodetype == "dir":
            return f"{str(self.path)}: {len(self.children)} items, {self.total_size()} bytes"
        else:
            return f"{str(self.path)}: {self.total_size()} bytes"
    
inode(path=PurePosixPath("/"))

/: 0 items, 0 bytes

In [6]:
cwd = PurePosixPath("/")
lspath = None

fs = {}

def mkdirp(path) -> inode:
    if str(path) in fs:
        # print(f"looked up {path}")
        pass
    elif len(path.parents) == 0:
        # print("creating root dir")
        node = inode(path=path, nodetype="dir")
        fs[str(path)] = node
    else:
        # print(f"creating {path}")
        parent = mkdirp(path.parent)
        node = inode(path=path)
        parent.children.append(node)
        fs[str(path)] = node
    return fs[str(path)]

mkdirp(PurePosixPath('/'))
fs

{'/': /: 0 items, 0 bytes}

In [7]:
for line in input_data:
    if line == "$ cd /":
        cwd = PurePosixPath("/")
    elif line == "$ cd ..":
        cwd = cwd.parent
    elif line.startswith("$ cd "):
        cwd = cwd / line[5:]
    elif line == "$ ls":
        lspath = cwd
    else:
        stat, name = line.split(" ")
        path = lspath / name
        
        if stat == "dir":
            node = mkdirp(path)
            node.nodetype = "dir"
            
        else:
            node = mkdirp(path)
            node.nodetype = "file"
            node.size = int(stat)

In [8]:
len(fs)

496

In [9]:
fs['/'].children

[/dpbwg: 3 items, 308482 bytes,
 /dvwfscw: 8 items, 7871401 bytes,
 /hccpl: 3 items, 14058385 bytes,
 /jsgbg: 1 items, 202682 bytes,
 /lhjmzsl: 1 items, 197308 bytes,
 /mwvbpw.mmg: 63532 bytes,
 /npj: 239480 bytes,
 /pngs: 4 items, 545639 bytes,
 /qhs: 2 items, 306017 bytes,
 /shvgmwn.vhv: 303649 bytes,
 /sjrrgd.phh: 236905 bytes,
 /sntcp: 13 items, 21647288 bytes,
 /sqs: 1 items, 252966 bytes]

In [10]:
fs['/'].total_size()

46233734

#### Part 1 Answer
Find all of the directories with a total size of at most 100000.  
**What is the sum of the total sizes of those directories?**

In [11]:
def small_dirs(thresh=100000):
    return [d for d in fs.values() if d.nodetype == "dir" and d.total_size() <= thresh]

len(small_dirs())

32

In [12]:
sum([d.total_size() for d in small_dirs()])

1350966

---

### Part 2

In [13]:
disk_size = 70000000
update_size = 30000000
available_size = disk_size - fs['/'].total_size()
size_needed = update_size - available_size
size_needed

6233734

#### Part 2 Answer

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update.  
**What is the total size of that directory?**

In [14]:
def big_dirs(threshold=size_needed):
    dirs = [d for d in fs.values() if d.nodetype == 'dir' and d.total_size() >= threshold]
    return sorted(dirs, key=lambda x: x.total_size())

big_dirs()

[/hccpl/qsgv/lhjmzsl: 7 items, 6296435 bytes,
 /dvwfscw: 8 items, 7871401 bytes,
 /sntcp/vqcvbncp: 9 items, 9470683 bytes,
 /hccpl/qsgv: 9 items, 12653650 bytes,
 /hccpl: 3 items, 14058385 bytes,
 /sntcp: 13 items, 21647288 bytes,
 /: 13 items, 46233734 bytes]